# 2.2. Differential expression analysis

We extracted differentially expressed genes for each successive pair of time points, up to 175 minutes (36/50 time points) as the last cycle was not meaningful according to the Kelliher's paper.

Differential expression analysis was done with the robust QLF test and the adjusted p-value was independently computed for each comparison.

## Input

* `data-create_networks/yeast_Kelliher_2016/yeast_WT_counts.txt`: raw counts generated by featureCounts from Kelliher 2016 dataset.

## Output

* `data-create_networks/yeast_Kelliher_2016/dea_quality.pdf`: quality control of the differential expression analysis.
* Files within the folder `data-create_networks/yeast_Kelliher_2016/comparisons`: results of the DEA for the 18 comparisons.
* `data-create_networks/yeast_Kelliher_2016/yeast_WT_logCPM.txt`: normalized counts in log CPM.

In [1]:
library("edgeR")

Loading required package: limma



In [2]:
count_file = '../../data-create_networks/yeast_Kelliher_2016/yeast_WT_counts.txt'
dea_quality_file = '../../data-create_networks/yeast_Kelliher_2016/dea_quality.pdf'
results_folder = '../../data-create_networks/yeast_Kelliher_2016/comparisons/'
logCPM_file = '../../data-create_networks/yeast_Kelliher_2016/yeast_WT_logCPM.txt'

## Import data

In [3]:
# import data
counts = read.table( count_file, skip = 1, header = TRUE, row.names = 1 )
counts = counts[ , 6:ncol(counts) ] # remove first columns

# rename header
colnames(counts) = paste((0:35)*5, 'min', sep='')

# concatenate the even and odd column headers
header = colnames(counts)
header = strsplit( header, 'min' )
even = seq(1, length(header), 2); odd = seq(2, length(header), 2)
reducedHeader = paste( header[even], header[odd], sep='-' )

# experiment design
N_replicates = dim(counts)[2]/2
group = factor( rep( 1:N_replicates, each=2 ), levels=1:N_replicates )
design = model.matrix( ~0+group )

## Prepare dataset

In [4]:
initCds = DGEList( counts, group = group )

# filter lowly expressed genes
keepGenes = filterByExpr( initCds )
initCds = initCds[ keepGenes, , keep.lib.sizes=FALSE ]

cat( "Number of genes before filtering:", nrow( counts ), "\n" )
cat( "Number of genes after filtering:", nrow( initCds$counts ), "\n" )

# compute model
initCds = calcNormFactors( initCds )
initCds = estimateDisp( initCds, design, robust = TRUE )
fit = glmQLFit( initCds, design, robust = TRUE )
cat( "BCV =", sqrt( initCds$common.dispersion ) )

Number of genes before filtering: 7126 
Number of genes after filtering: 6506 
BCV = 0.1078579

In [5]:
pdf(dea_quality_file)

plotMDS( initCds )
plotBCV( initCds )

cols = colnames(design)
qlfTests = list()
for( i in 1:(N_replicates - 1) ){
    # contrast
    comparison = paste( cols[i+1], cols[i], sep='-' )
    contrast = makeContrasts( comparison, levels=design )
    
    # test
    name = paste( reducedHeader[i+1], reducedHeader[i], sep="vs" )
    qlfTests[name] = list( glmQLFTest( fit, contrast = contrast ) )
    # adjusted p-value
    padj = p.adjust( qlfTests[[name]]$table$PValue, method = "BH" )
    qlfTests[[name]]$table$FDR = padj
    # display
    plotQLDisp( qlfTests[[name]], main=paste(name, "glm edgeR QL Dispersion") )
    hist(qlfTests[[name]]$table$PValue,150,main=paste(name, "glm edgeR tagwise dispersion"),xlab="raw pvalue")
}

dev.off()

png 
  2

In [6]:
### exports
i = 1
for( name in names(qlfTests) ){
    write.table( qlfTests[[name]]$table,
                 file=paste( results_folder, formatC(i, width=2, flag="0"), "_yeast_WT_",
                             name, ".txt", sep='' ),
                 sep = "\t", quote=FALSE, row.names=TRUE, col.names=NA )
    i = i + 1
}

write.table( cpm(initCds, prior.count=2, log=TRUE),
             file = logCPM_file,
             sep = "\t", quote=FALSE, row.names=TRUE, col.names=NA )